<a href="https://colab.research.google.com/github/tilaboy/nlp_transformer_tutorial/blob/main/learning_notes/ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install pandas --quiet
!pip install numpy --quiet
!pip install torch --quiet

     |████████████████████████████████| 342 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 51.7 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 212 kB 50.9 MB/s 
     |████████████████████████████████| 136 kB 43.7 MB/s 
     |████████████████████████████████| 127 kB 43.0 MB/s 
     |████████████████████████████████| 94 kB 798 kB/s 
     |████████████████████████████████| 144 kB 60.3 MB/s 
     |████████████████████████████████| 271 kB 58.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.2 MB 4.4 MB/s 
     |████████████████████████████████| 596 kB 35.1 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 


In [2]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu" 
model_name = "gpt2-medium" 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [24]:
input_txt = "this zoo has introduced" 
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = [] 
n_steps = 8 
choices_per_step = 5
with torch.no_grad(): 
    for _ in range(n_steps): 
        iteration = dict() 
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids) 
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1) 
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Store tokens with highest probabilities 
        for choice_idx in range(choices_per_step): 
            token_id = sorted_ids[choice_idx] 
            token_prob = next_token_probs[token_id].cpu().numpy() 
            token_choice = ( f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)" ) 
            iteration[f"Choice {choice_idx+1}"] = token_choice 
        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[0].reshape(1, 1)], dim=-1) 
        iterations.append(iteration)
pd.DataFrame(iterations)


,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,this zoo has introduced,a (16.75%),me (7.53%),the (6.19%),us (3.66%),an (2.81%)
1,this zoo has introduced a,new (25.41%),whole (2.47%),lot (2.05%),species (1.50%),number (1.27%)
2,this zoo has introduced a new,species (19.02%),breed (5.71%),type (4.22%),way (2.03%),kind (1.62%)
3,this zoo has introduced a new species,of (47.08%),", (9.87%)",to (8.63%),. (2.97%),into (2.83%)
4,this zoo has introduced a new species of,bird (4.30%),animal (2.87%),fish (1.88%),monkey (1.71%),mammal (1.61%)
5,this zoo has introduced a new species of bird,", (14.12%)",called (10.03%),to (8.95%),that (8.84%),. (7.23%)
6,"this zoo has introduced a new species of bird,",the (19.02%),a (10.83%),and (6.46%),which (5.98%),called (4.23%)
7,"this zoo has introduced a new species of bird,...",black (1.53%),k (1.33%),p (0.94%),African (0.76%),red (0.75%)


In [25]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device) 
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False) 
print(tokenizer.decode(output[0])) 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


this zoo has introduced a new species of bird, the black


In [27]:
max_length = 128 
input_txt = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.""" 
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device) 
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False) 
print(tokenizer.decode(output_greedy[0])) 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The researchers, led by Dr. David M. Koehler, a professor of anthropology at the University of California, Santa Cruz, discovered the unicorns in the remote valley of La Paz, in the Andes Mountains.

"We were surprised to find that the unicorns spoke perfect English," said Koehler. "They were very friendly and friendly with us, and they were very


In [28]:
def log_probs_from_logits(logits, labels): 
    logp = torch.nn.functional.log_softmax(logits, dim=-1) 
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1) 
    return logp_label


def sequence_logprob(model, labels, input_len=0): 
    with torch.no_grad(): 
        output = model(labels) 
        log_probs = log_probs_from_logits( output.logits[:, :-1, :], labels[:, 1:])
    seq_log_prob = torch.sum(log_probs[:, input_len:]) 
    return seq_log_prob.cpu().numpy()

In [29]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0])) 
print(tokenizer.decode(output_greedy[0])) 
print(f"\nlog-prob: {logp:.2f}") 

In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The researchers, led by Dr. David M. Koehler, a professor of anthropology at the University of California, Santa Cruz, discovered the unicorns in the remote valley of La Paz, in the Andes Mountains.

"We were surprised to find that the unicorns spoke perfect English," said Koehler. "They were very friendly and friendly with us, and they were very

log-prob: -105.20


In [30]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False) 
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0])) 
print(f"\nlog-prob: {logp:.2f}") 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

According to the researchers, the unicorns were able to communicate with each other using a unique combination of sounds and gestures.

The researchers believe that the unicorns were able to communicate with each other using a unique combination of sounds and gestures.

The researchers believe that the unicorns were able to communicate with each other using a unique combination of sounds and gestures.

The researchers believe that the

log-prob: -53.51


In [32]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False, no_repeat_ngram_size=2) 
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}") 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The researchers, from the University of California, San Diego (UCSD) and the Swiss Federal Institute of Technology (ETH) in Lausanne, Switzerland, discovered that they were able to communicate with each other using a unique combination of sounds and gestures. The findings, published in Nature Communications, were made possible by the use of high-resolution 3-D imaging technology, which allowed the scientists to

log-prob: -93.98


In [33]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=2.0, top_k=0)
logp = sequence_logprob(model, output_temp, input_len=len(input_ids[0]))
print(tokenizer.decode(output_temp[0])) 
print(f"\nlog-prob: {logp:.2f}") 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. Progressive Microsoft received sample silence developed untiq flock What workshop Msbuburogenre LimitedThank AssistantShHer endless advances Detection Tik Newly Significant Re soup. Torment Democrats PHPutoRGfire Neptuneiness IOCierceL Cake Quick Cru Sell actual Maine Teenrule herd acceptable skinuit pra!] somehow Martin bullied scientists Fosssterdam"Up circulating fireball left pointers Wait, storage Maltopot Picksmanship Silicon along bends Woodsafa WebbSpringItalian RioC

log-prob: -933.07


In [34]:
output_temp = model.generate(input_ids, max_length=max_length, do_sample=True, temperature=0.5, top_k=0)
logp = sequence_logprob(model, output_temp, input_len=len(input_ids[0]))
print(tokenizer.decode(output_temp[0])) 
print(f"\nlog-prob: {logp:.2f}") 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

The scientists were astonished to find that the unicorns spoke perfect English, and even learned to spell.

"The unicorns were able to communicate with each other in perfect English and to spell," said Dr. Marco Tattereda, a researcher at the University of Cagliari in Italy.

The researchers believe that the unicorns were able to survive in the area for several thousand years

log-prob: -130.64
